In [1]:
#1 Make question worse
import os
import json
import random
from helpers.make_query_worse import remove_fraction_function_words
from helpers.scholar_qa_pipeline import get_scholar_qa_answer
import time
import argparse
import numpy as np
import pandas as pd
from openai import OpenAI

torch not found, custom baseline rerankers will not work.


In [2]:
with open(os.path.join('data', 'human_answers.json'), 'r') as file:
    data = json.load(file)

In [ ]:
def get_response(_messages, _model='gpt-4.1', _max_tokens=300):

    client = OpenAI(
        api_key = open(os.path.join('../../PhD/apikeys', 'ai2_openai_key')).read().strip()
    )

    attempts = 0
    max_attempts = 3
    response = 'None'

    while attempts < max_attempts:
        try:
            response = client.chat.completions.create(
                model= _model,
                max_completion_tokens=_max_tokens,
                messages = _messages
            )
            
            _answer = response.choices[0].message.content.strip()
            print(f'{_answer}')
            answer_dict = json.loads(_answer)
            break

        except Exception as e:
            print(f'Error {e}. Sleeping 3 seconds ...')
            time.sleep(3)
            if attempts == max_attempts-1:
                answer_dict = f'Error {e}'

        attempts += 1
        time.sleep(0.5)

    return answer_dict

In [ ]:
def remove_clauses(original_query):
    
    print(f"Making Question Worse Version 2")
    
    system_prompt = '''You are a linguist. Given a question, split the question into clauses and rank them by how important they are for undestanding the question. Then remove the least important clause from the question.

Follow these steps:

    1. Read the query.
    2. Identify the noun phrases in the question.
    3. Rank them by importance for understanding the question.
    4. Remove the least important noun phrase from the question.
    5. Return the ranked clauses and modified question in the following JSON format: [{"1": "most important clause", "2": "second most important clauses", "3": "third most important clauses"}, {"worse_question": "modified question"}]. Do not provide any additional information or explanations, just the two JSON dictionaries.
'''

    user_prompt = f'''Original Query:\n{original_query}\n\n

JSON Output: {{"1": ""}}'''

    messages = [
                    {'role': 'developer', 'content': system_prompt},
                    {'role': 'user', 'content': user_prompt}
                ]

    response = get_response(messages, _model='gpt-4.1', _max_tokens=300)
    
    return response

In [ ]:
def remove_noun_phrases(original_query):
    
    print(f"Making Question Worse Version 2")
    
    system_prompt = '''You are a linguist. Given a question, split the question into noun phrases and rank them by how important they are for undestanding the question. Then remove the least important noun phrase from the question.

Follow these steps:

    1. Read the query.
    2. Identify the noun phrases in the question.
    3. Rank them by importance for understanding the question.
    4. Remove the least important noun phrase from the question.
    5. Return the ranked clauses and modified question in the following JSON format: [{"1": "most important clause", "2": "second most important clauses", "3": "third most important clauses"}, {"worse_question": "modified question"}]. Do not provide any additional information or explanations, just the two JSON dictionaries.
'''

    user_prompt = f'''Original Query:\n{original_query}\n\n

JSON Output: {{"1": ""}}'''

    messages = [
                    {'role': 'developer', 'content': system_prompt},
                    {'role': 'user', 'content': user_prompt}
                ]

    response = get_response(messages, _model='gpt-4.1', _max_tokens=300)
    
    return response

In [11]:
for d in data:
    query = d['input']
    # new_query = remove_fraction_function_words(query, fraction=0.7)
    new_query = remove_noun_phrases(query)
    print(f"Original Query: {query}",
          f"\nWorse Query: {new_query}", "\n")

Making Question Worse Version 2
{"1": "lipid nanoparticles (LNPs)", "2": "biomolecular corona", "3": "various serum proteins and blood components"}
{"worse_question": "What are the key mechanisms for lipid nanoparticles (LNPs) to form biomolecular corona with various serum proteins?"}
Error Extra data: line 2 column 1 (char 116). Sleeping 3 seconds ...
{"1": "lipid nanoparticles (LNPs)", "2": "biomolecular corona", "3": "various serum proteins and blood components"}
{"worse_question": "What are the key mechanisms for lipid nanoparticles (LNPs) to form biomolecular corona with various serum proteins?"}
Error Extra data: line 2 column 1 (char 116). Sleeping 3 seconds ...
{"1": "key mechanisms", "2": "lipid nanoparticles (LNPs)", "3": "biomolecular corona", "4": "various serum proteins", "5": "blood components"}
{"worse_question": "What are the key mechanisms for lipid nanoparticles (LNPs) to form biomolecular corona with various serum proteins?"}
Error Extra data: line 2 column 1 (char 1

UnboundLocalError: cannot access local variable 'answer_dict' where it is not associated with a value

In [5]:
new_answer.keys()

dict_keys(['sections', 'cost'])

In [18]:
new_answer['sections'][0]

{'title': 'Introduction to Biomolecular Corona Formation',
 'tldr': 'When lipid nanoparticles (LNPs) enter biological fluids like blood, biomolecules spontaneously adsorb onto their surface forming a "biomolecular corona." This corona fundamentally alters the biological identity and behavior of LNPs, affecting their targeting abilities, cellular interactions, and therapeutic efficacy. (8 sources)',
 'text': '\nUpon administration into the bloodstream, lipid nanoparticles (LNPs) undergo a crucial transformation as they interact with various biological components, particularly serum proteins. This interaction results in the rapid formation of a layer of biomolecules surrounding the nanoparticle, known as the "biomolecular corona" or more commonly the "protein corona" due to the predominant presence of plasma proteins <Paper corpusId="263607931" paperTitle="(Amici et al., 2023)" isShortName></Paper>. This corona formation is a spontaneous process that occurs through non-covalent interacti